In [ ]:
!pip install adafruit-circuitpython-  
!pip install numpy



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
## Join class for DH table to determine current position
import numpy as np

class joint:
    
    def __init__(self, theta, d, alpha, a):
        self.theta = theta
        self.alpha = alpha
        self.a = a
        self.d = d
    
    def A(self):
        return A_i(self.theta, self.alpha, self.a, self.d)

    def add_angle(self, theta = 0):
        self.theta = self.theta + theta

## ----------------------------------------------------------------------
# Basic functions for xarm

# Compute the A matrix for a single joint
def A_i(th_i, alpha_i, a_i, d_i):

    # np uses radians 
    th_i = np.deg2rad(th_i)
    alpha_i = np.deg2rad(alpha_i)

    #Create A matrix for 
    A = np.array([[np.cos(th_i), -np.sin(th_i)*np.cos(alpha_i), np.sin(th_i)*np.sin(alpha_i), a_i*np.cos(th_i)],
              [np.sin(th_i), np.cos(th_i)*np.cos(alpha_i), -np.cos(th_i)*np.sin(alpha_i), a_i*np.sin(th_i)], 
              [0, np.sin(alpha_i), np.cos(alpha_i), d_i],
              [0,0,0,1]])
    return A

# Multiply 6 joints Transform matricies together for the forward kinematics -- gives the matrix defining the 
#   Transformation from the end effector to the base frame -- should be in cartesian coordinates?
def multiply_all(joints):
    length = len(joints)
    # use np matmul to multiply all matricies one by one A1 * A2 * A3 * A4 * A5 * A6
    T = joints[0].A()
    for i in range(1,length):
        T = np.matmul(T,joints[i].A())
    
    
    return T



# grabs the last column of a matrix for extracting the cartesian coordinates, assuming the end effector is 
#   the final coordinate frame
#       Otherwise an offset computation is needed
def get_carte(A):
    cart = A[0:3,3]
    return cart




In [1]:
# create base joints from 
# these need to be changed when we calculate the angles right? 
# every time we have a new position we'd need to update this

#Joints from DH table
'''
Theta - rotation about z axis
d     - displacement along z axis
alpha - rotation about x axis
a     - translation along x axis

This is assuming the neutral position where the camera is at about level with the ground
    -- for looking around
    -- may need a neutral where we are looking at ground @ 45 degree angle
'''
# Theta, d_i, alpha, a -- degrees and mm

# from ping sensor to the base? tbd on this maybe assume not?
# or we just take x positive to be in line with ping sensor, then rotate it up the arm

# 90 degree rotation from horizontal to vertical up the arm in line with the base servo

# base_j = joint(90  , 0 ,  0 , -30)

# # from the base to the shoulder -- rotate forward or CW -> negative
# #   --shoulder servo angled forward -- to look horizontal
# shoulder_j = joint( -70, 0   , 0 , 45)
# # from the shoulder to the elbow
# #   --elbow servo angled up a bit
# elbow_j = joint(50, 0   , 0 , 65)
# #from elbow to wrist -- should end up about level. 
# #   --wrist angled to the max -- either 180 or 0 idr
# wrist_j = joint(-20, 0, 0  , 65)
# #from wrist to hand
# hand_j = joint(0  , 0   , 0  , 90)



ping2base_j = joint(180, 0, 0, 45)
base2shoulder_j = joint(-90  , 0 ,  0 , 70) # rotate back up and extend to the shoulder servo pivot

# from shoulder to the elbow  -- rotate forward or CW -> negative -- then out 65mm to the elbow pivot
#   --shoulder servo angled fbackward to angle zero -- not quite 90
shoulder2elbow_j = joint(88, 0   , 0 , 65)

# from the elbow to the wrist
#   --elbow servo angled up a bit
elbow2wrist_j = joint(57, 0   , 0 , 65)
#from elbow to wrist -- should end up about level. 
#   --wrist angled to the max -- either 180 or 0 idr
wrist2effector_j = joint(-64, 0, 0  , 90)
#from wrist to hand
# hand_j = joint(0  , 0   , 0  , 90)
# no sixth joint
# joint6 = joint(0  , 61.5 , 0 , 0)

#Change joint angles, joint 6 doesn't matter

shoulder2elbow_j.add_angle(-88) # straight up
elbow2wrist_j.add_angle(-57) # 
wrist2effector_j.add_angle(64)

# hand_j.add_angle(0.1) -- DNE -- just a 


all_joints = [ping2base_j, base2shoulder_j, shoulder2elbow_j, elbow2wrist_j, wrist2effector_j]
T06 = multiply_all(all_joints)
xyz = get_carte(T06)


#print("All joints matrix")
#print(T06)
print("End effector position (mm):")
print("X:"+ str(xyz[0]) + " Y:" + str(xyz[1]) + " Z:" + str(xyz[2]))

joint_angles = np.zeros(5)
for i in range(0,5):
    joint_angles[i] = all_joints[i].theta



NameError: name 'joint' is not defined

In [ ]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50  # standard servo frequency
kit = ServoKit(channels=16)  # For standard small servos

# -----------------------------
# Servo configuration
# -----------------------------
# Define which channels have large/high-torque vs small/RC servos
large_servos = [6,7]          # high-torque / metal gear
small_servos = [13, 14, 15] # standard hobby/RC servos

# Create servo objects for large servos with wide pulse range
large_servo_objects = {}
for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s
    s.angle = 0  # initialize to center
    time.sleep(0.05)

time.sleep(1)  # give all large servos time to reach position

# -----------------------------
# Helper functions
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    """
    Gradually move a large servo to the target angle
    """
    target = max(0, min(180, int(round(target))))
    if target == current:
        return current

    step = max(1, int(abs(step)))

    if target > current:
        angle = current
        while angle < target:
            angle = min(angle + step, target)
            servo_obj.angle = angle
            time.sleep(delay)
    else:
        angle = current
        while angle > target:
            angle = max(angle - step, target)
            servo_obj.angle = angle
            time.sleep(delay)

    return target  # updated current angle

def sweep_large_servo(ch):
    """
    Sweep a large servo on channel `ch` using gradual steps
    """
    s = large_servo_objects[ch]
    current_angle = 90
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 180, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)

def sweep_small_servo(ch):
    """
    Sweep a small hobby/RC servo using direct jumps
    """
    print(f"Sweeping small servo on channel {ch}")
    for angle in [0, 90, 180, 90, 0]:
        kit.servo[ch].angle = angle
        time.sleep(1)  # give servo time to reach position

# -----------------------------
# Main program
# -----------------------------
try:
    print("=== Sweeping LARGE servos ===")
    for ch in large_servos:
        print(f"Channel {ch}")
        sweep_large_servo(ch)

    print("\n=== Sweeping SMALL servos ===")
    for ch in small_servos:
        sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    pca.deinit()

In [ ]:
#Class Structure for robot arm
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit


class robo_arm:
    
    
    def __init__(self):
        # red cube vs blue cube vs green cube?
        # boolean for each? so we only pick up one of each. 
        ping2base_j = joint(180, 0, 0, 45)
        base2shoulder_j = joint(-90  , 0 ,  0 , 70) # rotate back up and extend to the shoulder servo pivot
        # from shoulder to the elbow  -- rotate forward or CW -> negative -- then out 65mm to the elbow pivot
        #   --shoulder servo angled fbackward to angle zero -- not quite 90
        shoulder2elbow_j = joint(88, 0   , 0 , 65)
        # from the elbow to the wrist
        #   --elbow servo angled up a bit
        elbow2wrist_j = joint(57, 0   , 0 , 65)
        #from elbow to wrist -- should end up about level. 
        #   --wrist angled to the max -- either 180 or 0 idr
        wrist2effector_j = joint(-64, 0, 0  , 90)


        # -----------------------------
        # PCA9685 setup
        # -----------------------------
        i2c = busio.I2C(board.SCL, board.SDA)
        pca = PCA9685(i2c)
        pca.frequency = 50  # standard servo frequency
        # kit = ServoKit(channels=16)  # For standard small servos

        large_servos = [7]          # high-torque / metal gear
        small_servos = [13, 14, 15] # standard hobby/RC servos

        self.kit = ServoKit(channels=16)
        super().__init__() # something along these lines

        # print(self.kit._items)
        # print(self.kit._pca)
        # print

      
        # Stops the update --not sure if this will work -- 11/17 unused
        self.stop = False 

        self.servo_angles_default = {
            "base" : 90, 
            "shoulder" : 180, # servo 5
            "elbow" : 0,  # servo
            "wrist" : 0,   # servo 
            "hand" : 180
        }

        # looks down towards ground at 45 degrees
        self.servo_angles_45down = {
            "base" : 0,  # -- check this
            "shoulder" : 90, # servo 5 -- check this  maybe slightly back from 90
            "elbow" : 180,  # servo
            "wrist" : 0,   # servo 
            "hand" : 180
        }
        
        self.servo_angles_safe = {
            "base" : 0,  # -- check this
            "shoulder" : 180, # servo 5 -- check this  maybe slightly back from 90
            "elbow" : 0,  # servo
            "wrist" : 180,   # servo 
            "hand" : 180
        }

        self.servo_angles_straight_up = {
            "base" : 0, 
            "shoulder" : 90, 
            "elbow" : 90,  # servo
            "wrist" : 90,   # servo 
            "hand" : 180
        }

        self.servo_angles_reach_forward = {
            "base" : 180, 
            "shoulder" : 180, # servo 5
            "elbow" : 90,  # servo
            "wrist" : 90,   # servo 
            "hand" : 180
        }

        self.servo_angles_elbow_l = {
            "base" : 0, 
            "shoulder" : 90, 
            "elbow" : 180,  # servo
            "wrist" : 90,   # servo 
            "hand" : 180
        }
        self.servo_angles_wrist_l = {
            "base" : 0, 
            "shoulder" : 90, 
            "elbow" : 90,  # servo
            "wrist" : 0,   # servo 
            "hand" : 180
        }
        ## ^^^^^^^^^^^^^^^ Angles mostly for Calibratio ^^^^^^^^^^^
        #set current angles to default
        self.current_servo_angles = self.servo_angles_default

        self.next_servo_angles = self.servo_angles_default
        #set default angles
        # self.servo_angles = {
        #     "base" : 90,
        #     "shoulder" : 0,
        #     "elbow" : 135,
        #     "wrist" : 90,
        #     "hand" : 180
        # }

        # self.servo_list = [wrist]
        self.servo_channel = {
        "base" : 0,
        "shoulder" : 7, # Servo 5
        "elbow" : 13,    # Servo 4
        "wrist" : 14,    # Servo 3
        "hand" : 15      # Servo 1
        }

        self.large_servo_objects = {}

        for servo_joint in self.servo_angles_default:


            try:
                
                ch = self.servo_channel[servo_joint]
                if(servo_joint == "shoulder"):
                    #we are a t big servo -- do it difff
                    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
                    self.large_servo_objects[ch] = s
                    s.angle = self.servo_angles_default["shoulder"]
                    print("shoulder initd")
                    time.sleep(0.05)
                    time.sleep(1)
                else:
                    # print(ch)
                    self.kit.servo[ch].angle = self.servo_angles_default[servo_joint] 
                    # Debugging
                    print("set channel to :")
                    print(self.kit.servo[ch].angle)
            except Exception:
                print("failed to update channel: "+ ch)
                pass

        # servo0 = adafruit_motor.servo.Servo(self.servo_channel["elbow"])
        print("ROBO-ARM INIT COMPLETE")    
    # END OF INIT
     
    def move_to(self, servo_obj, current, target, step=2, delay=0.02):
        """
        Gradually move a large servo to the target angle
        """
        target = max(0, min(180, int(round(target))))
        if target == current:
            return current

        step = max(1, int(abs(step)))

        if target > current:
            angle = current
            while angle < target:
                angle = min(angle + step, target)
                servo_obj.angle = angle
                time.sleep(delay)
        else:
            angle = current
            while angle > target:
                angle = max(angle - step, target)
                servo_obj.angle = angle
                time.sleep(delay)

        return target  # updated current angle 
    # # define new servo angle -- by keyword atm 
    # def set_angle(self, **servos):    
        
    #     for servo, angle in servos.items():
    
    #         try:
    #             ch = self.servo_channel[servo]
    #             print(ch)
    #             #Save the angle in object dict
    #             self.servo_angles[servo] = angle
    #             #Update that servo to be the angle
    #             kit.servo[ch].angle = self.servo_angles[angle]
    #         except Exception:
    #             pass
    
    # simple update for the angles we changed
    def update(self, delay = 0.5):
        #optional delay
        time.sleep(delay)

        #iterate through servos and update the channel
        for servo_joint in self.current_servo_angles:
            ch = self.servo_channel[servo_joint]
            # print("update channgel: ")
            # print(ch)
            # try:
            if(self.stop == False):
                    
                    #Mr christian code rapper // or "elbow"
                if(servo_joint == "shoulder"):
                    # TODO fix reduncadncy
                    s = self.large_servo_objects[ch]
                    current_angle = self.current_servo_angles[servo_joint]  
                    next_angle = self.next_servo_angles[servo_joint]

                    current_angle = self.move_to(s, current_angle, next_angle)
                    self.current_servo_angles[servo_joint] = current_angle
                    
                    # time.sleep(delay) # a bit extra in there for large servo
                #Standard servos
                else:
                    ch = self.servo_channel[servo_joint]
                    self.kit.servo[ch].angle = self.next_servo_angles[servo_joint]  
                    self.current_servo_angles[servo_joint] = self.next_servo_angles[servo_joint]
                    # Debugging
                    # print("set channel to :")
                    # print(ch)
                    print(self.kit.servo[ch].angle) 
            else:
                break
            
            # except Exception:
                print("exception thrown in update")
                print(servo_joint)
                pass
        print("Robo-Arm Angle Update Complete")

    # actual values from servo channel
    def get_servo_angles(self):

        return [self.kit.servo[self.servo_channel["base"]].angle, 
                # self.kit.servo[self.servo_channel["shoulder"]].angle, 
                self.kit.servo[self.servo_channel["elbow"]].angle,
                self.kit.servo[self.servo_channel["wrist"]].angle,
                self.kit.servo[self.servo_channel["hand"]].angle]

        # return self.servo_angles
        #easy pose /update -- just get the order right
    # compute the current position relative to ping sensor
    def get_x_y(self):
        ''' 
            Use the DH table joints and Add angle -- always theta -- to compute the current x/y coordinatae for the end effector --
                -- rather, where the end effector will CLOSE

            -   some calibration may be necessary
            -   


            shoulder -- 90 is straight up: Add 0 degrees to theta. 180 is forward: add -90 to theta. 0 is back: +90 to theta
                        servo: 0   -> joint 

                        servo maps + 90 degrees in theta, and extends ~65mm
            elbow    -- 
        j
        
        '''
        
        shoulder2elbow_j.add_angle( - self.current_servo_angles["shoulder"]) # approximate with what we've got. this one's range is decent
        elbow2wrist_j.add_angle( - self.kit.servo[self.servo_channel["elbow"]].angle) # 
        wrist2effector_j.add_angle(self.kit.servo[self.servo_channel["wrist"]].angle)
        
        all_joints = [ping2base_j, base2shoulder_j, shoulder2elbow_j, elbow2wrist_j, wrist2effector_j]
        T06 = multiply_all(all_joints)
        xyz = get_carte(T06)
        return xyz
        



    def pose(self, shoulder = 90, elbow = 120, wrist = 0):
        
        #shoulder is 7V and not used atm
        self.next_servo_angles["shoulder"] = shoulder
        self.next_servo_angles["elbow"] = elbow
        self.next_servo_angles["wrist"] = wrist
        # self.servo_angles["hand"] = hand
        self.update()

    #return to netrual -- update included
    def pose_neutral(self):
        self.next_servo_angles["shoulder"] = 180
        self.next_servo_angles["elbow"] = 135
        self.next_servo_angles["wrist"] = 45
        # self.next_servo_angles["hand"]  = 180
        self.update()

    '''
    #currently deprecated from the servo update 11/17
    def pose_ramp(self,shoulder, elbow, wrist):
        step_deg = 1
        step_sec = 0.02
        #determine the direction of travel for each limb

        # negative if set angle is less than current angle
        # shoulder_dir = self.servo_angles["shoulder"] - shoulder
        elbow_diff = elbow - self.servo_angles["elbow"]  
        wrist_diff = wrist - self.servo_angles["wrist"]
        
        while(
            # self.servo_angles["shoulder"] != shoulder + step
            # self.servo_angles["elbow"] < elbow
            abs(elbow_diff) - step_deg > 0 or abs(wrist_diff) - step_deg > 0
        ):
            if(elbow_diff > 0):
                #keep this 
                self.servo_angles["elbow"] = self.servo_angles["elbow"] + step_deg
                #Try directly setting to ease some jitter -- still does the update
                self.kit.servo[self.servo_channel["elbow"]].angle = self.servo_angles["elbow"] + step_deg
            elif(elbow_diff < 0):
                self.servo_angles["elbow"] = self.servo_angles["elbow"] - step_deg
                self.kit.servo[self.servo_channel["elbow"]].angle = self.servo_angles["elbow"] - step_deg

            if(wrist_diff > 0):
                self.servo_angles["wrist"] = self.servo_angles["wrist"] + step_deg
                self.kit.servo[self.servo_channel["wrist"]].angle = self.servo_angles["wrist"] + step_deg
            elif(wrist_diff < 0):
                self.servo_angles["wrist"] = self.servo_angles["wrist"] - step_deg
                self.kit.servo[self.servo_channel["wrist"]].angle = self.servo_angles["wrist"] - step_deg
            
            # don't update if we do it here manually
            # self.update(0)
            elbow_diff = elbow - self.servo_angles["elbow"]
            wrist_diff = wrist - self.servo_angles["wrist"]
            # time.sleep(step_sec)
    '''
    # Sets the robot to look down towards block
    def pose_45down(self):
        self.next_servo_angles = self.servo_angles_45down
        self.update()

    # def rotate(self, base = 0)
    
    #Set the grib strength, barely touching is default. 180 is wide open
    def grab(self, strength = 120):
        self.next_servo_angles["hand"] = strength
        self.update(0.5)
        time.sleep(0.5)

    #release grip to 100% open
    def release(self):
        self.servo_angles["hand"] = 180
        self.update()
    

    # always close out here -- won't drop!
    def pose_go_home(self):
        self.next_servo_angles = self.servo_angles_safe
        # self.next_servo_angles["hand"] = grip # default grab 120 atm
        self.update()

        

    # always close out here -- won't drop!
    # 90, 120, 0
    def pose_stand_up_look_forward(self, grip = 120):
        self.next_servo_angles = self.servo_angles_default
        self.next_servo_angles["hand"] = grip # default grab 120 atm
        self.update()

    # for comparing the DH table
    def pose_reach_forward(self):
        self.next_servo_angles = self.servo_angles_reach_forward
        self.update()

    # for comparing the DH table
    def pose_straight_up(self):
        self.next_servo_angles = self.servo_angles_straight_up
        self.update()

    def pose_elbow_L(self):
        self.next_servo_angles = self.servo_angles_elbow_l
        self.update()

    def pose_wrist_L(self):
        self.next_servo_angles = self.servo_angles_wrist_l
        self.update()
        
# rotate the base, rotate the wrist up
# look right, 
# if somethin's found: returns the angle of the base with the arm pointed at the detected object
# if nothing's found: returns 0
# ** Untested 11-17
def look_left(self):
    self.servo_angles["wrist"] = 45 # guessing
    self.cube_found = False

    step_size = 1 #? 2?
    # need a ramp function for the base -- and a stop function

    # while object not detected
    # rotate base left
    while(self.servo_angles["base"] + step_size < 180 and not cube_found):
        self.servo_angles["base"] = self.servo_angles["base"] + step_size

    if(cube_found):
        return self.kit.servo[self.servo_channel["base"]].angle
    else:
        # look back
        ''' or wherever midpoint is'''
        while(self.servo_angles["base"] - step_size > 90 and not cube_found):
            self.servo_angles["base"] = self.servo_angles["base"] + step_size

        if(cube_found):
            # small chance we catch it on the way back
            return self.kit.servo[self.servo_channel["base"]].angle
        else:
            # we're back at neutral. -- nothing found
            return 0
    self.next_servo_angles = self.servo_angles_default

# Guessing on the polarity of these
# Lifts the wrist up to look horizontally, looks right, 
# if somethin's found: returns the angle of the base with the arm pointed at the detected object
# if nothing's found: returns 0
# untested 11-17
def look_right(self):
    self.servo_angles["wrist"] = 45 # guessing
    self.cube_found = False

    step_size = 1 #? 2?
    # need a ramp function for the base -- and a stop function

    # while object not detected
    # rotate base left

    # Use this same logic to prevent overflow on the robo- ramp function
    # this will have to change for the 
    while(self.servo_angles["base"] - step_size > 0 and not cube_found):
        self.servo_angles["base"] = self.servo_angles["base"] - step_size

    if(self.cube_found):
        return self.kit.servo[self.servo_channel["base"]].angle
    else:
        # look back
        ''' or wherever midpoint is'''
        while(self.servo_angles["base"] + step_size < 90 and not self.cube_found):
            self.servo_angles["base"] = self.servo_angles["base"] + step_size

        if(self.cube_found):
            # small chance we catch it on the way back
            return self.kit.servo[self.servo_channel["base"]].angle
        else:
            # we're back at neutral. 
            return 0

# we need a way to continously track the object? 
# def track(self):
    # reference gets posted - a distance or an angle 
    # we minimize the difference between the reference and our value


In [ ]:
# create base joints from 
# these need to be changed when we calculate the angles right? 
# every time we have a new position we'd need to update this

#Joints from DH table
'''
Theta - rotation about z axis
d     - displacement along z axis
alpha - rotation about x axis
a     - translation along x axis

This is assuming the neutral position where the camera is at about level with the ground
    -- for looking around
    -- may need a neutral where we are looking at ground @ 45 degree angle
'''
# Theta, d_i, alpha, a -- degrees and mm

# from ping sensor to the base? tbd on this maybe assume not?
# or we just take x positive to be in line with ping sensor, then rotate it up the arm
# 90 degree rotation from horizontal to vertical up the arm in line with the base servo

base_j = joint(90  , 0 ,  0 , -30)
# from the base to the shoulder
#   --shoulder servo angled forward -- to look horizontal
shoulder_j = joint( -70, 0   , 0 , 45)
# from the shoulder to the elbow
#   --elbow servo angled up a bit
elbow_j = joint(50, 0   , 0 , 65)
#from elbow to wrist -- should end up about level. 
#   --wrist angled to the max -- either 180 or 0 idr
wrist_j = joint(-20, 0, 0  , 65)
#from wrist to hand
hand_j = joint(0  , 0   , 0  , 90)

# no sixth joint
# joint6 = joint(0  , 61.5 , 0 , 0)

#Change joint angles, joint 6 doesn't matter
shoulder_j.add_angle(0)
elbow_j.add_angle(23.3)
wrist_j.add_angle(80.1)

# hand_j.add_angle(0.1) -- DNE -- just a 


all_joints = [base_j, shoulder_j, elbow_j, wrist_j, hand_j]
T06 = multiply_all(all_joints)
xyz = get_carte(T06)


#print("All joints matrix")
#print(T06)
print("End effector position (mm):")
print("X:"+ str(xyz[0]) + " Y:" + str(xyz[1]) + " Z:" + str(xyz[2]))

joint_angles = np.zeros(5)
for i in range(0,5):
    joint_angles[i] = all_joints[i].theta




In [ ]:
# Determine the servo ranges
arm = robo_arm()
arm.pose(0,0,0)



In [ ]:
## Inverse Kinematics for Roboforge manipulator -- Assumes base is stationary -- 2x2 

In [ ]:
#Simple test for Arm HIL
arm = robo_arm()

print("Robo-Arm's Servo Channels used on breakout board")
print(arm.servo_channel)

print("Robo-Arm's Initial servo angles ")
print(arm.servo_angles)


#Simple pose -- snap right to angles
#angles are shoulder, elbow, wrist -- use grab to change hand angle
arm.pose(0, 0, 180)

arm.grab()

arm.pose_ramp(0,45,120)

arm.release() 


NameError: name 'robo_arm' is not defined

In [ ]:
# Sim for the arm -- print out the angles
# import robo_arm
arm = robo_arm(sim = True)
# print(arm.servo_angles)
arm.pose(0,0,0)
# print(arm.servo_angles)


ROBO-ARM INIT COMPLETE
{'base': 90, 'shoulder': 0, 'elbow': 135, 'wrist': 90, 'hand': 180}
Robo-Arm Angle Update Complete
{'base': 90, 'shoulder': 0, 'elbow': 0, 'wrist': 0, 'hand': 180}
